In [1]:
# !pip install opencv-python
# !pip install tensorflow
# !pip install matplotlib

import os
import cv2
# import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping


# import os
# import cv2
# # import pafy
# import math
# import random
# import numpy as np
# import datetime as dt
# import tensorflow as tf
# import keras
# from collections import deque
# import matplotlib.pyplot as plt


# from keras.layers import *
# from keras.models import Sequential
# from keras.utils import to_categorical
# from keras.utils import plot_model
# from keras.callbacks import EarlyStopping

from keras.models import load_model

# import keras

In [2]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 90
# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = ["normalHead", "oddHead"]

In [3]:
def frames_extraction(video_path):
    '''
    This function will extract the required frames from a video after resizing and normalizing them.
    Args:
        video_path: The path of the video in the disk, whose frames are to be extracted.
    Returns:
        frames_list: A list containing the resized and normalized frames of the video.
    '''
    # Declare a list to store video frames.
    frames_list = []
    # Read the Video File using the VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)
    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    print(video_frames_count)
    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count / SEQUENCE_LENGTH), 1)

    # Iterate through the Video Frames.
    for frame_counter in range(SEQUENCE_LENGTH):
        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        # Reading the frame from the video.
        success, frame = video_reader.read()
        # Check if Video frame is not successfully read then break the loop
        if not success:
            break
        # Resize the Frame to fixed height and width.

#         plt.imshow(frame)
#         plt.show()

        # cropped_image = frame[350:850,650:2]

        # plt.imshow(cropped_image)
        # plt.show()

        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        # plt.imshow(resized_frame)
        # plt.show()

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255
        # Append the normalized frame into the frames list
        frames_list.append(normalized_frame)
    # Release the VideoCapture object.
    video_reader.release()
    # Return the frames list.
    return frames_list

In [4]:
def create_dataset():
    '''
    This function will extract the data of the selected classes and create the required dataset.
    Returns:
        features:          A list containing the extracted frames of the videos.
        labels:            A list containing the indexes of the classes associated with the videos.
        video_files_paths: A list containing the paths of the videos in the disk.
    '''
    # Declared Empty Lists to store the features, labels and video file path values.
    features_train = []
    features_test = []

    labels_train = []
    labels_test = []

    video_files_paths = []

    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(CLASSES_LIST):
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')
        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(f'C:\\Users\\CSE-P07-2179\\Documents\\headmovement\\{class_name}')
        # Iterate through all the files present in the files list.
        for file_name in files_list:
            # Get the complete video path.
            video_file_path = os.path.join(f'C:\\Users\\CSE-P07-2179\\Documents\\headmovement\\{class_name}\\{file_name}')
            print(video_file_path)

            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)
            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:
                # Append the data to their repective lists.
                features_train.append(frames)
                labels_train.append(class_index)
                video_files_paths.append(video_file_path)


    for class_index, class_name in enumerate(CLASSES_LIST):
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')
        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(f'C:\\Users\\CSE-P07-2179\\Documents\\headmovement\\{class_name}Test')
        # Iterate through all the files present in the files list.
        for file_name in files_list:
            # Get the complete video path.
            video_file_path = os.path.join(f'C:\\Users\\CSE-P07-2179\\Documents\\headmovement\\{class_name}Test\\{file_name}')
            print(video_file_path)
            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)
            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:
                # Append the data to their repective lists.
                features_test.append(frames)
                labels_test.append(class_index)
                video_files_paths.append(video_file_path)

    # Converting the list to numpy arrays
    features_train = np.asarray(features_train)
    labels_train = np.array(labels_train)

    features_test = np.asarray(features_test)
    labels_test = np.array(labels_test)

    # Return the frames, class index, and video file path.
    return features_train,features_test, labels_train,labels_test, video_files_paths

In [6]:
features_train, features_test, labels_train, labels_test, video_files_paths = create_dataset()
print(np.shape(features_train))
print(np.shape(features_test))

labels_train = to_categorical(labels_train)
labels_test = to_categorical(labels_test)

Extracting Data of Class: normalHead
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_10_N_1.mp4
64
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_10_N_2.mp4
81
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_10_N_3.mp4
59
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_10_N_4.mp4
41
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_10_N_5.mp4
160
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_1.mp4
41
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_10.mp4
88
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_11.mp4
83
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_2.mp4
84
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_3.mp4
100
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_4.mp4
32
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_5.mp4
56
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_6.mp4
71
C:\Users\C

C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_2_N_10.mp4
120
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_2_N_2.mp4
31
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_2_N_3.mp4
21
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_2_N_4.mp4
79
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_2_N_5.mp4
122
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_2_N_6.mp4
89
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_2_N_7.mp4
0
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_2_N_8.mp4
65
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_2_N_9.mp4
56
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_30_N_1.mp4
156
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_30_N_2.mp4
166
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_30_N_3.mp4
151
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_30_N_4.mp4
34
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead

C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_17_C_1.mp4
36
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_17_C_10.mp4
253
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_17_C_11.mp4
25
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_17_C_12.mp4
82
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_17_C_13.mp4
16
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_17_C_2.mp4
83
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_17_C_3.mp4
73
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_17_C_4.mp4
73
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_17_C_5.mp4
64
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_17_C_6.mp4
49
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_17_C_7.mp4
135
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_17_C_8.mp4
32
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_17_C_9.mp4
53
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_18_C_1.mp4
5
C:\Users\CSE-P0

C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_31_C_7.mp4
353
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_31_C_8.mp4
-1
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_31_C_9.mp4
299
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_34_C_1.mp4
288
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_34_C_2.mp4
599
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_34_C_3.mp4
259
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_34_C_4.mp4
349
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_34_C_5.mp4
222
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_34_C_6.mp4
121
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_34_C_7.mp4
412
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_34_C_8.mp4
270
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_35_C_1.mp4
298
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_35_C_2.mp4
232
C:\Users\CSE-P07-2179\Documents\headmovement\oddHead\HM_35_C_3.mp4
239
C:\User

C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_37_N_6.mp4
273
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_8_N_1.mp4
65
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_8_N_2.mp4
23
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_8_N_3.mp4
63
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_8_N_4.mp4
47
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_8_N_5.mp4
121
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_8_N_6.mp4
-1
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_8_N_7.mp4
30
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_8_N_8.mp4
49
Extracting Data of Class: oddHead
C:\Users\CSE-P07-2179\Documents\headmovement\oddHeadTest\HM_12_C_1.mp4
61
C:\Users\CSE-P07-2179\Documents\headmovement\oddHeadTest\HM_12_C_10.mp4
26
C:\Users\CSE-P07-2179\Documents\headmovement\oddHeadTest\HM_12_C_11.mp4
22
C:\Users\CSE-P07-2179\Documents\headmovement\oddHeadTes

In [ ]:
def create_LRCN_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''

    # We will use a Sequential model for model construction.
    model = Sequential()

    # Define the Model Architecture.
    ########################################################################################################################

    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same', activation='relu'),
                              input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))

    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    # model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(Flatten()))

    model.add(LSTM(32))

    model.add(Dense(len(CLASSES_LIST), activation='softmax'))

    ########################################################################################################################

    # Display the models summary.
    model.summary()

    # Return the constructed LRCN model.
    return model

In [7]:
# LRCN_model = create_LRCN_model()

# File path
filepath = "./LRCN_model___Date_Time_2021_12_10__13_58_06___Loss_1.2796132564544678___Accuracy_0.7021276354789734.h5"

# # Load the model
LRCN_model=load_model(filepath)

In [65]:
# Get the loss and accuracy from model_evaluation_history.
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

# Define the string date format.
# Get the current Date and Time in a DateTime Object.
# Convert the DateTime object to string according to the style mentioned in date_time_format string.
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

# Define a useful name for our model to make it easy for us while navigating through multiple saved models.
model_file_name = f'LRCN_model___Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.h5'

# Save the Model.
LRCN_model.save(model_file_name)

model_evaluation_accuracy

0.6399999856948853

In [18]:
optimizer = tf.keras.optimizers.Adam(lr=0.001)
LRCN_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy"])

# Start training the model.
LRCN_model_training_history = LRCN_model.fit(x=features_train, y=labels_train, epochs=10, batch_size=2, shuffle=True,
                                             validation_split=0.2)

model_evaluation_history = LRCN_model.evaluate(features_test, labels_test)

Epoch 1/10
64/64 [==============================] - 12s 153ms/step - loss: 0.6715 - accuracy: 0.6172 - val_loss: 1.0810 - val_accuracy: 0.0000e+00
Epoch 2/10
64/64 [==============================] - 8s 130ms/step - loss: 0.6688 - accuracy: 0.6172 - val_loss: 1.0067 - val_accuracy: 0.0000e+00
Epoch 3/10
64/64 [==============================] - 8s 132ms/step - loss: 0.6673 - accuracy: 0.6172 - val_loss: 0.9924 - val_accuracy: 0.0000e+00
Epoch 4/10
64/64 [==============================] - 8s 130ms/step - loss: 0.6662 - accuracy: 0.6172 - val_loss: 0.9751 - val_accuracy: 0.0000e+00
Epoch 5/10
64/64 [==============================] - 8s 130ms/step - loss: 0.6664 - accuracy: 0.6172 - val_loss: 0.9618 - val_accuracy: 0.0000e+00
Epoch 6/10
64/64 [==============================] - 8s 132ms/step - loss: 0.6662 - accuracy: 0.6172 - val_loss: 0.9676 - val_accuracy: 0.0000e+00
Epoch 7/10
64/64 [==============================] - 8s 133ms/step - loss: 0.6675 - accuracy: 0.6172 - val_loss: 0.9542 - va

In [42]:
def create_convlstm_model():
    '''
    This function will construct the required convlstm model.
    Returns:
        model: It is the required constructed convlstm model.
    '''

    # We will use a Sequential model for model construction
    model = Sequential()

    # Define the Model Architecture.
    ########################################################################################################################
    
    model.add(ConvLSTM2D(filters = 4, kernel_size = (3, 3), activation = 'tanh',data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True, input_shape = (SEQUENCE_LENGTH,
                                                                                      IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 8, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 14, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    #model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(Flatten()) 
    
    model.add(Dense(len(CLASSES_LIST), activation = "sigmoid"))
    
    ########################################################################################################################
     
    # Display the models summary.
    model.summary()
    
    # Return the constructed convlstm model.
    return model

In [43]:
# Construct the required convlstm model.
convlstm_model = create_convlstm_model()

# Display the success message. 
print("Model Created Successfully!")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 45, 62, 62, 4)     1024      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 45, 31, 31, 4)    0         
 )                                                               
                                                                 
 time_distributed_36 (TimeDi  (None, 45, 31, 31, 4)    0         
 stributed)                                                      
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 45, 29, 29, 8)     3488      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 45, 15, 15, 8)    0         
 3D)                                                             
                                                      

In [44]:
# Create an Instance of Early Stopping Callback
# early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
optimizer = keras.optimizers.Adam(lr=0.01)
# Compile the model and specify loss function, optimizer and metrics values to the model
convlstm_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

# Start training the model.
convlstm_model_training_history = convlstm_model.fit(x = features_train, y = labels_train, epochs = 50, batch_size = 4,
                                                     shuffle = True, validation_split = 0.2, validation_data=[x,y]
                                                     )

Epoch 1/50
55/55 [==============================] - 128s 2s/step - loss: 0.7235 - accuracy: 0.5571 - val_loss: 0.7668 - val_accuracy: 0.0000e+00
Epoch 2/50
55/55 [==============================] - 121s 2s/step - loss: 0.6868 - accuracy: 0.5845 - val_loss: 0.8425 - val_accuracy: 0.0000e+00
Epoch 3/50
55/55 [==============================] - 122s 2s/step - loss: 0.6823 - accuracy: 0.5799 - val_loss: 0.9311 - val_accuracy: 0.0545
Epoch 4/50
55/55 [==============================] - 124s 2s/step - loss: 0.6564 - accuracy: 0.5936 - val_loss: 0.7481 - val_accuracy: 0.3818
Epoch 5/50
55/55 [==============================] - 131s 2s/step - loss: 0.6534 - accuracy: 0.6119 - val_loss: 1.0924 - val_accuracy: 0.0182
Epoch 6/50
55/55 [==============================] - 133s 2s/step - loss: 0.6046 - accuracy: 0.6393 - val_loss: 1.7004 - val_accuracy: 0.3091
Epoch 7/50
55/55 [==============================] - 134s 2s/step - loss: 0.5900 - accuracy: 0.6804 - val_loss: 2.1209 - val_accuracy: 0.0727
Epoch

In [53]:
# Get the loss and accuracy from model_evaluation_history.
model_evaluation_history = convlstm_model.evaluate(features_test, labels_test)
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

# Define the string date format.
# Get the current Date and Time in a DateTime Object.
# Convert the DateTime object to string according to the style mentioned in date_time_format string.
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

# Define a useful name for our model to make it easy for us while navigating through multiple saved models.
model_file_name = f'convlstm_model___Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.h5'

# Save your Model.
convlstm_model.save(model_file_name)

3/3 [==============================] - 5s 2s/step - loss: 3.1245 - accuracy: 0.6400
